
$$
r(s_t,a_t)=\mathbb I(s_t=[\text{EOS}])r(x,y)-\beta KL(t)
$$

- RLHF 中的 reward 定义在 response 级别，而非中间的 token 级别
    - 中间的 token reward 默认为 0
    - reward + kl-penalty
- Advantage
    - $\delta_t^V=r_t+\gamma V_\theta(s_{t+1})-V_\theta(s_t)$
    - $A_\theta^{GAE}(s_t,a)=\delta_t^V+\gamma\lambda A_\theta^{GAE}(s_{t+1},a)$
- returns = advantages + values
    - δt = rt + γ·V(st+1) - V(st)
    - At = δt + (γλ)At+1


```python
def compute_gae_advantage_return(token_level_rewards: torch.Tensor, values: torch.Tensor, eos_mask: torch.Tensor, gamma: torch.Tensor, lam: torch.Tensor):
    with torch.no_grad():
        lastgaelam = 0
        advantages_reversed = []
        gen_len = token_level_rewards.shape[-1]

        for t in reversed(range(gen_len)):
            nextvalues = values[:, t + 1] if t < gen_len - 1 else 0.0
            delta = token_level_rewards[:, t] + gamma * nextvalues - values[:, t]
            lastgaelam = delta + gamma * lam * lastgaelam
            advantages_reversed.append(lastgaelam)
        advantages = torch.stack(advantages_reversed[::-1], dim=1)

        returns = advantages + values
        advantages = verl_F.masked_whiten(advantages, eos_mask)
    return advantages, returns
```